In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from google.cloud import storage
from google.cloud.storage.bucket import Bucket

from initial import parse_company_page, post_process_company_df, parse_round_page, post_process_round_df, parse_investor_page, post_process_investor_df, parse_acquisition_page, post_process_acquisition_df, map_company_status, map_acq_amount_to_num
import pandas as pd
from tqdm.notebook import tqdm
from utils import get_base_domain, cleanup_prefectur_info, get_usd, split2row_with_index, json_col_to_df, export_to_s3, validate_datetime_str_format, validate_date_str_format
from cb import generate_cb_uuid
from datetime import datetime
from loguru import logger


pd.options.display.max_columns = 999

In [3]:
client = storage.Client()
bucket: Bucket = client.get_bucket("initial-htmls")

In [17]:
blobs = bucket.list_blobs(prefix="initial.inc/investors")
blob_cnt = sum(1 for _ in blobs)

blobs = bucket.list_blobs(prefix="initial.inc/investors")
investor_dicts = []

for blob in tqdm(blobs, total=blob_cnt):
    html_byte = blob.download_as_string()
    my_dict = parse_investor_page(html_byte.decode('utf-8'))
    investor_dicts.append({**my_dict, "Company url": '/' + '/'.join(blob.name.split("/")[-3:-1]), "_timestamp": blob.name.split("/")[-1]})

investor_df = pd.DataFrame(investor_dicts)
investor_df = investor_df[investor_df['Company url'] != '']
print(len(investor_df))
investor_df.head()

  0%|          | 0/8502 [00:00<?, ?it/s]

8502


,Company name,website,address,founded date,kinds,attributes,alias,snss,Company url,_timestamp
0,"Hokkaido Venture Capital, Inc.",http://www.hokkaido-vc.com/,"NCO Sapporo Station North Exit 2F, 2-20 Kitash...",August 1999,VCs,Independent,"HVC Co., Ltd. (former)",[],/investors/1,1679992252661
1,"Resona Capital Co., Ltd.",https://www.resona-gr.co.jp/resonacapital/,"Fukagawa Gatharia Tower S Building 16F, 1-5-25...",March 1988,VCs,"Banks, credit unions, credit unions",,[],/investors/10,1679992252661
2,ITOCHU Finance Corporation,http://www.itcfnc.co.jp/,"2-5-1 Kita-Aoyama, Minato-ku, Tokyo Itochu Bui...",November 1993,VCs,trading company,,[],/investors/1008,1679992252661
3,"TNP Partners, Corporation",http://www.tnp-g.jp,"Shin-Yokohama SR Building 8F, 3-6-1 Shin-Yokoh...",September 2000,VCs,Independent,"TSUNAMI Network Partners Co., Ltd. (former)",[],/investors/1013,1679992252661
4,"Knowledge Company Co., Ltd.",http://www.knowledgeco.jp/,,,,,,[],/investors/1015,1679992252661
